In [1]:
!nvidia-smi

Fri Jan  5 17:25:21 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A40          On   | 00000000:D8:00.0 Off |                    0 |
|  0%   30C    P8    28W / 300W |      0MiB / 45634MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import argparse
import os
import numpy as np
import pandas as pd
import torch
import yaml
from torch.utils.data import DataLoader
#from utils import *
import matplotlib.pyplot as plt
import torch.nn.utils.spectral_norm as spectralnorm
from torch.nn import init
import torchvision
import torch
import torch.nn as nn
from models import *
from utils import *
import torch.nn.functional as F
from helper import *

cuda = True
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

import argparse

In [3]:
def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
    alpha = Tensor(np.random.random((real_samples.size(0),  1, 1, 1)))
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = D(interpolates)
    fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    # Get gradient w.r.t. interpolates
    gradients = autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake[:,0],
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

def cal_W1(ssf, discriminator, discriminator_M, test_loader):
    mse_loss = nn.MSELoss(reduction='sum')

    set_models_state([ssf, discriminator, discriminator_M], 'eval')

    W1_distance = []
    W1M_distance = []
    MSE = []

    num_x = 0
    for i, x in enumerate(iter(test_loader)):
        with torch.no_grad():
            #Get the data
            x = x.permute(0, 4, 1, 2, 3)
            x = x.cuda().float()
            x_cur = x[:,:,1,...]
            x_ref = x[:,:,0,...]
            x_hat = ssf(x_cur, x_ref)


            fake_vid = torch.cat((x[:,:,0,:,:], x_hat), dim = 1).detach()
            real_vid = x[:,0,:2,...].detach() #this looks good!

            fake_validity = discriminator(fake_vid)
            real_validity = discriminator(real_vid)

            fake_img = x_hat.detach()
            real_img = x[:,0,6:7,...].detach()
            fake_valid_m = discriminator_M(fake_img)
            real_valid_m = discriminator_M(real_img)

            W1_distance.append(torch.sum(real_validity) - torch.sum(fake_validity))
            W1M_distance.append(torch.sum(real_valid_m) - torch.sum(fake_valid_m))
            #print (F.mse_loss(x[:,:,1,:,:], x_hat)* x.size()[0])
            MSE.append(mse_loss(x[:,:,1,:,:], x_hat))
            #print (mse_loss(x[:,:,1,:,:], x_hat)/(64*64*len(x)))
            num_x += len(x)

    W1_distance = torch.Tensor(W1_distance)
    W1M_distance = torch.Tensor(W1M_distance)
    MSE = torch.Tensor(MSE)

    return W1M_distance.sum()/num_x, W1_distance.sum()/num_x, MSE.sum()/(64*64*num_x)

def set_models_state(list_models, state):
    if state =='train':
        for model in list_models:
            model.train()
    else:
        for model in list_models:
            model.eval()

def set_opt_zero(opts):
    for opt in opts:
        opt.zero_grad()


In [4]:
def main(
    dim =128,
    z_dim=1,
    lambda_gp=10,
    bs=64,
    d_penalty=0,
    skip_fq=5,
    total_epochs=100,
    lambda_P=0,
    lambda_PM=0,
    lambda_MSE=1,
    L = 2,
    path='./data/',
    pre_path='None',
    single_bit=False):
  
    
    #Create folder:
    folder_name='dim_'+str(dim)+'|z_dim_'+str(z_dim)+'|L_'+str(L)+'|_single_bit'+str(single_bit)+'|lambda_gp_'+str(lambda_gp) \
        +'|bs_'+str(bs)+'|dpenalty_'+str(d_penalty)+'|lambdaP_'+str(lambda_P)+'|lambdaPM_'+str(lambda_PM)+'|lambdaMSE_' + str(lambda_MSE)
    print ("Settings: ", folder_name)

    os.makedirs('./saved_models/'+ folder_name, exist_ok=True)
    f = open('./saved_models/'+ folder_name + "/performance.txt", "a")

    #Define Models
    discriminator = Discriminator_v3(out_ch=2) #Generator Side
    discriminator_M = Discriminator_v3(out_ch=1) #Marginal Discriminator
    ssf = ScaleSpaceFlow(num_levels=1, dim=z_dim, stochastic=True, quantize_latents=True, L=L,single_bit=single_bit)

    list_models = [discriminator, discriminator_M, ssf]

    ssf.cuda()
    discriminator.cuda()
    discriminator_M.cuda()

    #Load models:
    if pre_path != 'None':
        print('Using pretrained weights!')
        #prefix_path = 'z'+str(z_dim)+'l'+str(L)+'_MMSE'
        ssf.motion_encoder.load_state_dict(torch.load(pre_path+'/m_enc.pth'))
        ssf.motion_decoder.load_state_dict(torch.load(pre_path+'/m_dec.pth'))
        ssf.P_encoder.load_state_dict(torch.load(pre_path+'/p_enc.pth'))
        ssf.res_encoder.load_state_dict(torch.load(pre_path+'/r_enc.pth'))
        ssf.res_decoder.load_state_dict(torch.load(pre_path+'/r_dec.pth'))

        discriminator.load_state_dict(torch.load(pre_path+'/discriminator.pth'))
        discriminator_M.load_state_dict(torch.load(pre_path+'/discriminator_M.pth'))

    #Define Data Loader
    train_loader, test_loader = get_dataloader(data_root=path, seq_len=8, batch_size=bs, num_digits=1)
    mse = torch.nn.MSELoss()

    #discriminator.train()
    opt_ssf= torch.optim.RMSprop(ssf.parameters(), lr=5e-5)
    opt_d = torch.optim.RMSprop(discriminator.parameters(), lr=5e-5)
    opt_dm = torch.optim.RMSprop(discriminator_M.parameters(), lr=5e-5)

    list_opt = [opt_ssf, opt_d, opt_dm]

    for epoch in range(total_epochs):
        print(epoch)
        set_models_state(list_models, 'train')
        for i,x in enumerate(iter(train_loader)):
            #Set 0 gradient
            set_opt_zero(list_opt)

            #Get the data
            x = x.permute(0, 4, 1, 2, 3)
            x = x.cuda().float()
            x_cur = x[:,:,1,...]
            x_ref = x[:,:,0,...]
            x_hat = ssf(x_cur, x_ref)

            #Optimize discriminator
            fake_vid = torch.cat((x[:,:,0,:,:], x_hat), dim = 1)
            real_vid = x[:,0,:2,...].detach() #this looks good!
            fake_validity = discriminator(fake_vid.detach())
            real_validity = discriminator(real_vid)
            gradient_penalty = compute_gradient_penalty(discriminator, real_vid.data, fake_vid.data)
            errVD =  -torch.mean(real_validity) + torch.mean(fake_validity) + lambda_gp * gradient_penalty
            errVD.backward()
            opt_d.step()

            #Optimize discriminator M
            fake_img = x_hat.detach()
            real_img = x[:,0,:1,...].detach()
            fake_valid_m = discriminator_M(fake_img)
            real_valid_m = discriminator_M(real_img)
            gradient_penalty_m = compute_gradient_penalty(discriminator_M, fake_img.data, real_img.data)
            errID =  -torch.mean(real_valid_m) + torch.mean(fake_valid_m) + lambda_gp * gradient_penalty_m
            errID.backward()
            opt_dm.step()

            if i%skip_fq == 0:
                x_cur = x_cur.detach()
                x_ref = x_ref.detach()
                x_hat = ssf(x_cur, x_ref)

                fake_vid = torch.cat((x_ref, x_hat), dim = 1)
                fake_validity = discriminator(fake_vid)
                errVG = -torch.mean(fake_validity)

                fake_img = x_hat
                fake_validity_im = discriminator_M(fake_img)
                errIG = -torch.mean(fake_validity_im)

                loss = lambda_MSE*mse(x_hat, x_cur) + lambda_P*errVG +  lambda_PM*errIG
                loss.backward()

                opt_ssf.step()

        if epoch %10 == 0:
            show_str= "Epoch: "+ str(epoch) + "l_PM, l_P, l_MSE, d_penalty " + str(lambda_PM) + str(lambda_P)+ " " \
            +str(lambda_MSE) + " " + str(d_penalty) + " P loss: " + str(cal_W1(ssf, discriminator, discriminator_M, test_loader))
            print (show_str)

            f.write(show_str+"\n")

    show_str= "Epoch: "+ str(epoch) + "l_PM, l_P, l_MSE, d_penalty " + str(lambda_PM) + str(lambda_P)+ " " \
            +str(lambda_MSE) + " " + str(d_penalty) + " P loss: " + str(cal_W1(ssf, discriminator, discriminator_M, test_loader))
    print (show_str)
    f.write(show_str+"\n")

    set_models_state(list_models, 'eval')

    torch.save(ssf.motion_encoder.state_dict(), os.path.join("./saved_models/" + folder_name, 'm_enc.pth'))
    torch.save(ssf.motion_decoder.state_dict(), os.path.join("./saved_models/" + folder_name, 'm_dec.pth'))
    torch.save(ssf.P_encoder.state_dict(), os.path.join("./saved_models/" + folder_name, 'p_enc.pth'))
    torch.save(ssf.res_encoder.state_dict(), os.path.join("./saved_models/" + folder_name, 'r_enc.pth'))
    torch.save(ssf.res_decoder.state_dict(), os.path.join("./saved_models/" + folder_name, 'r_dec.pth' ))
    torch.save(discriminator.state_dict(), os.path.join("./saved_models/" + folder_name, 'discriminator.pth'))
    torch.save(discriminator_M.state_dict(), os.path.join("./saved_models/" + folder_name, 'discriminator_M.pth'))

    f.close()

    #save some figures
    for i,x in enumerate(iter(train_loader)):
        x = x.permute(0, 4, 1, 2, 3)
        x = x.cuda().float()
        break
    np.savez_compressed("./saved_models/" + folder_name+"/x", a=x.detach().cpu().numpy())

    for i in range(5): #generate same figure 5 times
        x_cur = x[:,:,1,...]
        x_ref = x[:,:,0,...]
        x_hat = ssf(x_cur, x_ref)
        np.savez_compressed("./saved_models/" + folder_name+"/x_hat"+str(i), a=x_hat.detach().cpu().numpy())


In [10]:
#training just on MSE
pre_path = './saved_models/dim_128|z_dim_1|L_2|_single_bitFalse|lambda_gp_10|bs_64|dpenalty_0|lambdaP_0|lambdaPM_0|lambdaMSE_1'

main(
    z_dim = 1,
    L = 2,
    total_epochs = 150,
    lambda_P=0,
    lambda_PM=0,
    lambda_MSE=1,
    skip_fq=1
    #,pre_path = pre_path
)

Settings:  dim_128|z_dim_1|L_2|_single_bitFalse|lambda_gp_10|bs_64|dpenalty_0|lambdaP_0|lambdaPM_0|lambdaMSE_1
[-1.0, 1.0]
Finished Loading MNIST!
0
Epoch: 0l_PM, l_P, l_MSE, d_penalty 00 1 0 P loss: (tensor(8.4095), tensor(8.1159), tensor(0.0153))
1
2
3
4
5
6
7
8
9
10
Epoch: 10l_PM, l_P, l_MSE, d_penalty 00 1 0 P loss: (tensor(4.7515), tensor(4.6795), tensor(0.0095))
11
12
13
14
15
16
17
18
19
20
Epoch: 20l_PM, l_P, l_MSE, d_penalty 00 1 0 P loss: (tensor(4.4079), tensor(4.4672), tensor(0.0091))
21
22
23
24
25
26
27
28
29
30
Epoch: 30l_PM, l_P, l_MSE, d_penalty 00 1 0 P loss: (tensor(4.5189), tensor(4.4731), tensor(0.0088))
31
32
33
34
35
36
37
38
39
40
Epoch: 40l_PM, l_P, l_MSE, d_penalty 00 1 0 P loss: (tensor(4.4828), tensor(4.4368), tensor(0.0089))
41
42
43
44
45
46
47
48
49
50
Epoch: 50l_PM, l_P, l_MSE, d_penalty 00 1 0 P loss: (tensor(4.1799), tensor(4.0574), tensor(0.0087))
51
52
53
54
55
56
57
58
59
60
Epoch: 60l_PM, l_P, l_MSE, d_penalty 00 1 0 P loss: (tensor(4.2443), tensor

In [5]:
pre_path = './saved_models/dim_128|z_dim_1|L_2|_single_bitFalse|lambda_gp_10|bs_64|dpenalty_0|lambdaP_0|lambdaPM_0|lambdaMSE_1'
main(
    z_dim = 1,
    L = 2,
    total_epochs = 300,
    lambda_P=40e-3,
    lambda_PM=0,
    lambda_MSE=1,
    skip_fq=5,
    pre_path = pre_path
)

Settings:  dim_128|z_dim_1|L_2|_single_bitFalse|lambda_gp_10|bs_64|dpenalty_0|lambdaP_0.04|lambdaPM_0|lambdaMSE_1
[-1.0, 1.0]
Using pretrained weights!
Finished Loading MNIST!
0
Epoch: 0l_PM, l_P, l_MSE, d_penalty 00.04 1 0 P loss: (tensor(0.4378), tensor(0.3956), tensor(0.0146))
1
2
3
4
5
6
7
8
9
10
Epoch: 10l_PM, l_P, l_MSE, d_penalty 00.04 1 0 P loss: (tensor(0.5321), tensor(0.3761), tensor(0.0153))
11
12
13
14
15
16
17
18
19
20
Epoch: 20l_PM, l_P, l_MSE, d_penalty 00.04 1 0 P loss: (tensor(0.5231), tensor(0.3929), tensor(0.0152))
21
22
23
24
25
26
27
28
29
30
Epoch: 30l_PM, l_P, l_MSE, d_penalty 00.04 1 0 P loss: (tensor(0.5169), tensor(0.3882), tensor(0.0165))
31
32
33
34
35
36
37
38
39
40
Epoch: 40l_PM, l_P, l_MSE, d_penalty 00.04 1 0 P loss: (tensor(0.5279), tensor(0.3860), tensor(0.0208))
41
42
43
44
45
46
47
48
49
50
Epoch: 50l_PM, l_P, l_MSE, d_penalty 00.04 1 0 P loss: (tensor(0.6022), tensor(0.4304), tensor(0.0212))
51
52
53
54
55
56
57
58
59
60
Epoch: 60l_PM, l_P, l_MSE, d